In [ ]:
!pip install rdkit-pypi pandas requests


In [ ]:
import pandas as pd

from scripts.compute_descriptors import compute_descriptors, lipinski_rule_of_five
from scripts.admet_prediction import get_admet_predictions


In [ ]:
data = {
    "Drug": [
        "Chloroquine", 
        "Hydroxychloroquine", 
        "Amodiaquine", 
        "Artemisinin", 
        "Lumefantrine"
    ],
    "SMILES": [
        "CCN(CC)CCCC(C)Nc1ccnc2cc(Cl)ccc12",
        "CCN(CC)CCCC(C)Nc1ccnc2cc(Cl)ccc12O",
        "CCNc1ccc(OCCN(CC)CC)nc2c(Cl)cccc12",
        "CC1CCC2C(C1)C(OO2)C(=O)O",
        "CC(C)(C)OC1=C(C=C(C=C1)OC2CCCC2)OCCN3CCN(CC3)C"
    ]
}

df = pd.DataFrame(data)
df


In [ ]:
descriptor_rows = []

for i, row in df.iterrows():
    desc = compute_descriptors(row["SMILES"])
    if desc:
        desc["Drug"] = row["Drug"]
        desc["Lipinski"] = lipinski_rule_of_five(desc)
        descriptor_rows.append(desc)

desc_df = pd.DataFrame(descriptor_rows)
desc_df


In [ ]:
admet_rows = []

for i, row in df.iterrows():
    pred = get_admet_predictions(row["SMILES"])
    if pred:
        pred["Drug"] = row["Drug"]
        admet_rows.append(pred)

admet_df = pd.DataFrame(admet_rows)
admet_df


In [ ]:
final_df = desc_df.merge(admet_df, on="Drug", how="left")
final_df


In [ ]:
final_df.to_csv("../data/final_output.csv", index=False)
desc_df.to_csv("../data/descriptors_only.csv", index=False)
admet_df.to_csv("../data/admet_only.csv", index=False)

print("Files saved to data/ folder")


In [ ]:
summary = final_df[["Drug", "Lipinski", "HIA", "BBB", "Hepatotoxicity", "hERG_Block"]]
summary
